### Raw data

In [ ]:
"""Run all functions that load and cache data from remote."""

from build_dataset.workers import load_sensible_data as lsd


tc0 = {'hours': range(24),
    'days': range(7),
    'spans': [
        ("01/01/14", "31/12/14")
    ]
}
tc1 = {
    'hours': range(24),
    'days': range(7),
    'spans': [
        ("06/01/14", "24/01/14"),
        ("03/02/14", "16/05/14"),
        ("01/09/14", "05/12/14"),
        ("02/06/14", "20/06/14")
    ]
}
tc2 = {
    'hours': range(24),
    'days': range(7),
    'spans': [
        ("17/05/14", "01/06/14"),
        ("06/12/14", "21/12/14")
    ]
}
tc3 = {'hours': range(24),
    'days': range(7),
    'spans': [
        ("01/01/14", "05/01/14"),
        ("25/01/14", "02/02/14"),
        ("14/04/14", "20/04/14"),
        ("21/06/14", "30/08/14"),
        ("22/12/14", "31/12/14")
    ]
}

for i, tc in enumerate([tc0, tc1, tc2, tc3]):

    print "\n\n## ---------------- ##"
    print "## Building for tc%d ##" % i
    print "## ---------------- ##"

    print "\nLocation Reference..."
    print "\nsuccess!\n"
    print "Timezone Reference..."
    print "\nsuccess!\n"

    print "Iterating over datasets:"
    for dataset in ["calllog", "sms", "screen", "stop_locations", "bluetooth"]:
        print "\tBuilding '%s'...\t\t" % dataset,
        if dataset == "bluetooth":
            lsd.load(tc, dataset, filtering="bt_special", load_cached=False)
        else:
            pass
        print "\n\tsuccess!\n"

### Records

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
from collections import Counter
import os
from build_dataset.workers import load_sensible_data as lsd
from build_dataset.analysis import location_reference as locref

tc0 = {
    'hours': range(24),
    'days': range(7),
    'spans': [
        ("01/01/14", "31/12/14")
    ]
}

location_reference_tc0 = locref.Load_location_reference(tc0).location_reference

df_call = lsd.load(tc0, "calllog")
df_text = lsd.load(tc0, "sms")
df_screen = lsd.load(tc0, "screen")
df_stops = lsd.load(tc0, "stop_locations")
df_physical = lsd.load(tc0, "bluetooth")

users = set(list(df_call['user'])) & set(list(df_text['user'])) & set(list(df_screen['user'])) & \
        set(list(df_stops['user'])) & set(list(df_physical['user']))
    
def call_timer(func):
    """Simple decorator to time function calls."""
    def wrapper(*args):
        print func.__name__,
        start = dt.now()
        output = func(*args)
        print "time:", dt.now() - start
        return output
    return wrapper
    
@call_timer
def _filter_call(df_u):
    df_u = df_u.drop('user', 1)
    df_u.loc[:, 'interaction'] = "call"
    df_u.loc[:, 'timestamp'] = [dt.fromtimestamp(t) for t in df_u.loc[:, 'timestamp']]
    df_u.loc[:, 'type'] = ["in" if t == 1 else "out" for t in df_u.loc[:, 'type']]
    df_u.columns = ["duration", "correspondent_id", "datetime", "direction", "interaction"]
    return df_u

@call_timer
def _filter_text(df_u):
    df_u = df_u[df_u.loc[:, 'status'] <= 0]
    df_u = df_u[df_u.loc[:, 'type'] <= 2]
    df_u = df_u.drop(['status','user'], 1)
    df_u.loc[:, 'interaction'] = 'text'
    df_u.loc[:, 'timestamp'] = [dt.fromtimestamp(t) for t in df_u.loc[:, 'timestamp']]
    df_u.loc[:, 'type'] = ["in" if t == 1 else "out" for t in df_u.loc[:, 'type']]
    df_u.columns = ["correspondent_id", "datetime", "direction", "interaction"]
    return df_u

@call_timer
def _filter_physical(u, df_u):
    df_u.loc[:, 'interaction'] = 'physical'
    df_u.loc[:, 'timestamp'] = [dt.fromtimestamp(t) for t in df_u.loc[:, 'timestamp']]
    df_u.loc[:, 'correspondent_id'] = [a if a != u else b 
                                for a,b in zip(df_u.loc[:, 'bt_mac'], df_u.loc[:, 'user'])]
    df_u = df_u.drop(['class','id','bt_mac','rssi', 'user'], 1)
    df_u.columns = ["datetime", "interaction", "correspondent_id"]
    return df_u

@call_timer
def _filter_screen(df_u):
    sessions = []
    i = 0
    for row in df_u.iterrows():
        event, times = row[1]['screen_on'], row[1]['timestamp']
        if i == 0:
            prev_event = event
            prev_times = times
            i+=1; continue
        elif event == 0 and prev_event == 1:
            duration = int(times-prev_times)
            sessions.append({'datetime': dt.fromtimestamp(prev_times), 
                             'duration': duration})
        prev_event = event
        prev_times = times
        i+=1
    df_u = pd.DataFrame(sessions)
    df_u.loc[:, 'interaction'] = 'screen'
    return df_u

@call_timer
def _filter_stops(u, df_u):
    def evaluate_event(r):
        user, label, arrival, departure = \
        r[1]['user'], r[1]['label'], r[1]['arrival'], r[1]['departure']
        
        state = location_reference_tc0[str(user)][str(label)]
        if state['type'] == "home":
            return "home"
        if state['type'] == "campus":
            if state['__friday_bar'] and dt.fromtimestamp(np.mean([arrival,departure])).hour >=17:
                return "friday_bar"
            return "campus"
        return "other"
        
    df_u.loc[:, 'duration'] = df_u.loc[:, 'departure'] - df_u.loc[:, 'arrival']
    df_u.loc[:, 'datetime'] = [dt.fromtimestamp(t) for t in df_u.loc[:, 'arrival']]
    df_u.loc[:, 'position'] = ["%d_%s" % (u, l) for l in df_u.loc[:, 'label']]
    df_u.loc[:, 'event'] = [evaluate_event(r) for r in df_u.iterrows()]
    df_u.loc[:, 'interaction'] = 'stop'
    df_u = df_u.drop(['arrival', 'departure', 'label', 'lat', 'lon', 'timestamp', 'user'], 1)
    return df_u


def processed_users(tc):
    interaction_types = ["call", "text", "physical", "screen", "stop"]
    counter = Counter()
    for t in interaction_types:
        users = [int(f.split(".")[0]) for f in os.listdir("build_dataset/data_cache/records/%s/%s" % (t, tc))]
        counter.update(users)
    return [k for k,v in counter.items() if v == 5]
    

tc = "tc0"
skip_users = processed_users(tc)
for u in users:
    
    if u in skip_users:
        continue
    
    print "\nuser:", u
    try:
        df_call_u = _filter_call(df_call[df_call.loc[:, 'user'] == u])
        df_text_u = _filter_text(df_text[df_text.loc[:, 'user'] == u])
        df_physical_u = _filter_physical(u, df_physical.loc[(df_physical.loc[:, 'user'] == u) | \
                                                            (df_physical.loc[:, 'bt_mac'] == u), :])
        df_screen_u = _filter_screen(df_screen[df_screen.loc[:, 'user'] == u])
        df_stop_u = _filter_stops(u, df_stops[df_stops.loc[:, 'user'] == u])
    
        df_call_u.to_csv("build_dataset/data_cache/records/call/%s/%d.csv" % (tc, u), index=False)
        df_text_u.to_csv("build_dataset/data_cache/records/text/%s/%d.csv" % (tc, u), index=False)
        df_physical_u.to_csv("build_dataset/data_cache/records/physical/%s/%d.csv" % (tc, u), index=False)
        df_screen_u.to_csv("build_dataset/data_cache/records/screen/%s/%d.csv" % (tc, u), index=False)
        df_stop_u.to_csv("build_dataset/data_cache/records/stop/%s/%d.csv" % (tc, u), index=False)
    except:
        print "Failed:", u
    


user: 490
_filter_call time: 0:00:01.066041
_filter_text time: 0:00:00.348884
_filter_physical

/opt/conda/envs/python2/lib/python2.7/site-packages/pandas/core/indexing.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/opt/conda/envs/python2/lib/python2.7/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


 time: 0:00:12.941832
_filter_screen time: 0:00:01.339109
_filter_stops Failed: 490

user: 491
_filter_call time: 0:00:00.009415
_filter_text time: 0:00:00.030504
_filter_physical time: 0:00:01.893200
_filter_screen time: 0:00:03.168137
_filter_stops time: 0:00:01.512441

user: 492
_filter_call time: 0:00:00.008389
_filter_text time: 0:00:00.038854
_filter_physical time: 0:00:00.745294
_filter_screen time: 0:00:01.604188
_filter_stops time: 0:00:01.279393

user: 493
_filter_call time: 0:00:00.006597
_filter_text time: 0:00:00.014700
_filter_physical time: 0:00:00.672634
_filter_screen time: 0:00:00.888842
_filter_stops time: 0:00:01.317025

user: 494
_filter_call time: 0:00:00.003082
_filter_text time: 0:00:00.009008
_filter_physical time: 0:00:00.793035
_filter_screen time: 0:00:03.009673
_filter_stops time: 0:00:05.438939

user: 495
_filter_call time: 0:00:00.013646
_filter_text time: 0:00:00.012004
_filter_physical time: 0:00:00.644746
_filter_screen time: 0:00:01.039992
_filter_sto